In [2]:
!pip install pyld

  Obtaining dependency information for pyld from https://files.pythonhosted.org/packages/44/cd/80760be197a4bd08e7c136ef4bcb4a2c63fc799d8d91f4c177b21183135e/PyLD-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict from https://files.pythonhosted.org/packages/29/98/27e145ff7e8e63caa95fb8ee4fc56c68acb208bef01a89c3678a66f9a34d/frozendict-2.4.6-cp310-cp310-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 538.5 kB/s eta 0:00:00a 0:00:01


In [8]:
# Example script (Python-style pseudocode; adapt imports to your environment)

import json
from jsonschema import validate, ValidationError
from pyld import jsonld          # JSON-LD processing library
from rdflib import Graph        # RDF library

# 1) Define a JSON-LD context mapping to ontology URIs
context = {
    "@context": {
        "name": "http://schema.org/name",
        "knows": "http://schema.org/knows",
        "Person": "http://schema.org/Person"
    }
}

# 2) Define a JSON Schema that includes an @context → OWL class mapping
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "Person",
    "type": "object",
    "properties": {
        "@context": {
            "const": context["@context"],
            "description": "Ontology context mapping"
        },
        "@type": {
            "type": "string",
            "const": "Person",
            "description": "Must map to botlab.com/ont#Person"
        },
        "name": {
            "type": "string",
            "description": "Person's name"
        }
    },
    "required": ["@context", "@type", "name"]
}

# 3) Create a JSON-LD instance
person = {
    "@context": context["@context"],
    "@type": "Person",
    "name": "Alice",
    "knows": [
        {"@type": "Person", "name": "Bob"}
    ]
}

# 4) Validate against JSON Schema
try:
    validate(instance=person, schema=schema)
    print("✅ JSON validated successfully.")
except ValidationError as e:
    print("❌ Validation error:", e)

# 5) Expand and convert to TTL (outline; install and import pyld & rdflib in your env)
expanded = jsonld.expand(person)
g = Graph().parse(data=json.dumps(expanded), format="json-ld")
ttl = g.serialize(format="turtle")
print("--- Generated Turtle ---")
print(ttl)


✅ JSON validated successfully.
--- Generated Turtle ---
@prefix ns1: <http://schema.org/> .

[] a ns1:Person ;
    ns1:knows [ a ns1:Person ;
            ns1:name "Bob" ] ;
    ns1:name "Alice" .


